# **Notebook 10.1: 1D Convolution**

This notebook investigates 1D convolutional layers.

Work through the cells below, running each cell in turn. In various places you will see the words "TODO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.


NOTE!!

If you have the first edition of the printed book, it mistakenly refers to a convolutional filter with no spaces between the elements (i.e. a normal filter without dilation) as having dilation zero.  Actually, the convention is (weirdly) that this has dilation one.  And when there is one space between the elements, this is dilation two.   This notebook reflects the correct convention and so will be out of sync with the printed book.  If this is confusing, check the [errata](https://github.com/udlbook/udlbook/blob/main/UDL_Errata.pdf) document.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Define a signal that we can apply convolution to
x = [5.2, 5.3, 5.4, 5.1, 10.1, 10.3, 9.9, 10.3, 3.2, 3.4, 3.3, 3.1]

In [ ]:
# Draw the signal
fig,ax = plt.subplots()
ax.plot(x, 'k-')
ax.set_xlim(0,11)
ax.set_ylim(0, 12)
plt.show()

In [1]:
def conv_3_1_1_zp(x_in, omega):
    x_out = np.zeros_like(x_in)
    N = len(x_in)

    # Zero padded convolution with kernel size 3
    for i in range(N):
        acc = 0.0

        # Left neighbor
        if i - 1 >= 0:
            acc += omega[0] * x_in[i - 1]
        else:
            acc += omega[0] * 0

        # Center (i)
        acc += omega[1] * x_in[i]

        # Right neighbor
        if i + 1 < N:
            acc += omega[2] * x_in[i + 1]
        else:
            acc += omega[2] * 0

        x_out[i] = acc

    return x_out

Now let's see what kind of things convolution can do
First, it can average nearby values, smoothing the function:

In [ ]:

omega = [0.33,0.33,0.33]
h = conv_3_1_1_zp(x, omega)

# Check that you have computed this correctly
print(f"Sum of output is {np.sum(h):3.3}, should be 71.1")

# Draw the signal
fig,ax = plt.subplots()
ax.plot(x, 'k-',label='before')
ax.plot(h, 'r-',label='after')
ax.set_xlim(0,11)
ax.set_ylim(0, 12)
ax.legend()
plt.show()

Notice how the red function is a smoothed version of the black one as it has averaged adjacent values.  The first and last outputs are considerably lower than the original curve though.  Make sure that you understand why!<br><br>

With different weights, the convolution can be used to find sharp changes in the function:

In [ ]:

omega = [-0.5,0,0.5]
h2 = conv_3_1_1_zp(x, omega)

# Draw the signal
fig,ax = plt.subplots()
ax.plot(x, 'k-',label='before')
ax.plot(h2, 'r-',label='after')
ax.set_xlim(0,11)
# ax.set_ylim(0, 12)
ax.legend()
plt.show()

Notice that the convolution has a peak where the original function went up and trough where it went down.  It is roughly zero where the function is locally flat.  This convolution approximates a derivative. <br> <br>

Now let's define the convolutions from figure 10.3.  

In [ ]:
def conv_3_2_1_zp(x_in, omega):
    # Output size is ceil(N/2), already created above in boilerplate
    x_out = np.zeros(int(np.ceil(len(x_in) / 2)))
    N = len(x_in)

    # Stride = 2
    for out_i in range(len(x_out)):
        # Compute starting input index for this output position
        i = out_i * 2

        acc = 0.0

        # Left neighbor (i-1) — zero if out of range
        if i - 1 >= 0:
            acc += omega[0] * x_in[i - 1]
        else:
            acc += omega[0] * 0

        # Middle (i)
        if i < N:
            acc += omega[1] * x_in[i]
        else:
            acc += omega[1] * 0

        # Right neighbor (i+1)
        if i + 1 < N:
            acc += omega[2] * x_in[i + 1]
        else:
            acc += omega[2] * 0

        x_out[out_i] = acc

    return x_out

In [ ]:
omega = [0.33,0.33,0.33]
h3 = conv_3_2_1_zp(x, omega)

# If you have done this right, the output length should be six and it should
# contain every other value from the original convolution with stride 1
print(h)
print(h3)

In [2]:
def conv_5_1_1_zp(x_in, omega):
    x_out = np.zeros_like(x_in)
    N = len(x_in)

    for i in range(N):
        acc = 0.0

        # center offset from -2 to +2
        for k in range(-2, 3):
            idx = i + k
            if 0 <= idx < N:   # in bounds
                acc += omega[k+2] * x_in[idx]
            else:             # zero padding
                acc += omega[k+2] * 0

        x_out[i] = acc

    return x_out

In [ ]:

omega2 = [0.2, 0.2, 0.2, 0.2, 0.2]
h4 = conv_5_1_1_zp(x, omega2)

# Check that you have computed this correctly
print(f"Sum of output is {np.sum(h4):3.3}, should be 69.6")

# Draw the signal
fig,ax = plt.subplots()
ax.plot(x, 'k-',label='before')
ax.plot(h4, 'r-',label='after')
ax.set_xlim(0,11)
ax.set_ylim(0, 12)
ax.legend()
plt.show()

In [3]:
# Finally let's define a zero-padded convolution operation
# with a convolution kernel size of 3, a stride of 1, and a dilation of 2
# as in figure 10.3d.  Write it yourself, don't call a library routine!
# Don't forget that Python arrays are indexed from zero, not from 1 as in the book figures
def conv_3_1_2_zp(x_in, omega):
    x_out = np.zeros_like(x_in)
    N = len(x_in)

    for i in range(N):
        acc = 0.0

        # Dilation=2 → indices: i-2, i, i+2
        for k in [-2, 0, 2]:
            idx = i + k

            if 0 <= idx < N:          # valid index
                if k == -2:
                    acc += omega[0] * x_in[idx]
                elif k == 0:
                    acc += omega[1] * x_in[idx]
                else: # k == 2
                    acc += omega[2] * x_in[idx]
            else:
                # zero padding
                if k == -2:
                    acc += omega[0] * 0
                elif k == 0:
                    acc += omega[1] * 0
                else:
                    acc += omega[2] * 0

        x_out[i] = acc

    return x_out

In [ ]:
omega = [0.33,0.33,0.33]
h5 = conv_3_1_2_zp(x, omega)

# Check that you have computed this correctly
print(f"Sum of output is {np.sum(h5):3.3}, should be 68.3")

# Draw the signal
fig,ax = plt.subplots()
ax.plot(x, 'k-',label='before')
ax.plot(h5, 'r-',label='after')
ax.set_xlim(0,11)
ax.set_ylim(0, 12)
ax.legend()
plt.show()

Finally, let's investigate representing convolutions as full matrices, and show we get the same answer.

In [ ]:
def get_conv_mat_3_1_1_zp(n_out, omega):
    omega_mat = np.zeros((n_out, n_out))

    # Kernel offsets for dilation=1: [-1, 0, +1]
    offsets = [-1, 0, 1]

    for i in range(n_out):            # row index = output position
        for k, off in enumerate(offsets):
            j = i + off               # input index
            if 0 <= j < n_out:        # inside bounds
                omega_mat[i, j] = omega[k]
            else:                     # zero padded
                omega_mat[i, j % n_out] += 0   # no-op, here for clarity

    return omega_mat

In [ ]:
# Run original convolution
omega = np.array([-1.0,0.5,-0.2])
h6 = conv_3_1_1_zp(x, omega)
print(h6)

# If you have done this right, you should get the same answer
omega_mat = get_conv_mat_3_1_1_zp(len(x), omega)
h7 = np.matmul(omega_mat, x)
print(h7)


TODO:  What do you expect to happen if we apply the last convolution twice?  Can this be represented as a single convolution?  If so, then what is it?

If we apply the last convolution twice, we convolve the signal with the same 3-tap, gap-2 filter two times.